In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline
plt.style.use('ggplot')
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, graclus
from torch_cluster import graclus_cluster

In [3]:
df2011raw = pd.read_csv('E:\Github\MLN\FinalData/2011raw.csv')
dfgraph = pd.read_csv('E:\Github\MLN\FinalData/GraphProp.csv')
dfcensus = pd.read_csv('E:\Github\MLN\FinalData/census.csv')

In [24]:
df2001raw = pd.read_csv('E:\Github\MLN\FinalData/2001.csv')

In [4]:
df2011raw.head()

,Area Name,Last residence,Total migrants Persons,Total migrants Males,Total migrantsFemales,Work/employment Persons,Work/employment Males,Work/employment Females,Business Persons,Business Males,...,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370,Unnamed: 16371,Unnamed: 16372,Unnamed: 16373,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377
0,jammu & kashmir,jammu & kashmir,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,jammu & kashmir,himachal pradesh,8609,2688,5921,726,643,83,109,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jammu & kashmir,punjab,43938,12318,31620,3804,3237,567,816,708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jammu & kashmir,chandigarh,611,249,362,88,73,15,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jammu & kashmir,uttarakhand,3180,1633,1547,447,390,57,37,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df2011raw.shape

(1223, 16378)

In [6]:
dfgraph.head()

,Id,Label,timeset,modularity_class,Authority,Hub,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,weighted indegree,weighted outdegree,Weighted Degree,indegree,outdegree,Degree,pageranks,clustering,eigencentrality,Pop Size
0,andaman & nicobar islands,NaN,NaN,2,0.171440,0.171473,1,1.0,1.0,0.641716,81267,20700,101967,34,34,68,0.028992,0.983066,1.00000,380581.0
1,andhra pradesh,NaN,NaN,2,0.171440,0.171473,1,1.0,1.0,0.641716,1591890,2030004,3621894,34,34,68,0.028992,0.983066,1.00000,84580777.0
2,arunachal pradesh,NaN,NaN,1,0.167185,0.171600,1,1.0,1.0,0.422139,136010,37368,173378,33,34,67,0.028180,0.983957,0.97261,1383727.0
3,assam,NaN,NaN,1,0.171440,0.171473,1,1.0,1.0,0.641716,495699,659694,1155393,34,34,68,0.028992,0.983066,1.00000,31205576.0
4,bihar,NaN,NaN,0,0.171440,0.171473,1,1.0,1.0,0.641716,1111954,7453803,8565757,34,34,68,0.028992,0.983066,1.00000,104099452.0


In [7]:
dfcensus.head()

,State,Capitals,Type,Population,perctotalpopulation,Males,Females,SexRatio,Literacyrate,Rural Population,Urban Population,Area,Density,DecadalGrowth,Language
0,Andaman and Nicobar Islands,Port Blair,UT,380581,0.03,"202,871","177,710",876,86.63,"244,411","135,533","8,249",46,6.7%,Other
1,Andhra Pradesh,Hyderabad,State,84580777,6.99,"42,442,146","42,138,631",993,67.02,"56,361,702","28,219,075","275,045",308,10.98%,Other
2,Arunachal Pradesh,Itanagar,State,1383727,0.11,"713,912","669,815",938,65.38,"1,069,165","313,446","83,743",17,25.9%,Other
3,Assam,Dispur,State,31205576,2.58,"15,939,443","15,266,133",958,72.19,"26,780,526","4,388,756","78,438",397,16.9%,Assames
4,Bihar,Patna,State,104099452,8.60,"54,278,157","49,821,295",918,61.80,"92,075,028","11,729,609","94,163","1,102",25.1%,Bhojpuri


In [8]:
for i in range(34):
    print('{:28}  {}'.format(dfcensus['State'].tolist()[i], dfgraph.Id.tolist()[i]))

Andaman and Nicobar Islands   andaman & nicobar islands
Andhra Pradesh                andhra pradesh
Arunachal Pradesh             arunachal pradesh
Assam                         assam
Bihar                         bihar
Chandigarh                    chandigarh
Chhattisgarh                  chhattisgarh
Dadra and Nagar Haveli        dadra & nagar haveli
Daman and Diu                 daman & diu
Goa                           goa
Gujarat                       gujarat
Haryana                       haryana
Himachal Pradesh              himachal pradesh
Jammu and Kashmir             jammu & kashmir
Jharkhand                     jharkhand
Karnataka                     karnataka
Kerala                        kerala
Lakshadweep                   lakshadweep
Madhya Pradesh                madhya pradesh
Maharashtra                   maharashtra
Manipur                       manipur
Meghalaya                     meghalaya
Mizoram                       mizoram
Nagaland                      nagalan

In [9]:
dfgraph['Pop Size'] = dfcensus['Population']

In [10]:
dfgraph = dfgraph.drop(columns=['Label', 'timeset'])

In [11]:
dfgraph.columns

Index(['Id', 'modularity_class', 'Authority', 'Hub', 'Eccentricity',
       'closnesscentrality', 'harmonicclosnesscentrality',
       'betweenesscentrality', 'weighted indegree', 'weighted outdegree',
       'Weighted Degree', 'indegree', 'outdegree', 'Degree', 'pageranks',
       'clustering', 'eigencentrality', 'Pop Size'],
      dtype='object')

In [12]:
dffeatures = dfgraph[['weighted indegree', 'weighted outdegree','indegree', 'outdegree', 'Degree', 'pageranks','clustering', 'Pop Size']]
dffeatures[['Area','Density','DecadalGrowth']] = dfcensus[['Area','Density','DecadalGrowth']]

C:\Users\Alok Kumar\Python\Python37\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
dffeatures.head()

,weighted indegree,weighted outdegree,indegree,outdegree,Degree,pageranks,clustering,Pop Size,Area,Density,DecadalGrowth
0,81267,20700,34,34,68,0.028992,0.983066,380581,"8,249",46,6.7%
1,1591890,2030004,34,34,68,0.028992,0.983066,84580777,"275,045",308,10.98%
2,136010,37368,33,34,67,0.028180,0.983957,1383727,"83,743",17,25.9%
3,495699,659694,34,34,68,0.028992,0.983066,31205576,"78,438",397,16.9%
4,1111954,7453803,34,34,68,0.028992,0.983066,104099452,"94,163","1,102",25.1%


In [14]:
decadalgrowth = []

for i in dffeatures['DecadalGrowth'].str[:-1].tolist():
    try:
        decadalgrowth.append(np.float(i))
    except:
        decadalgrowth.append(-1 * float(i[1:]))

In [15]:
dffeatures['DecadalGrowth'] = decadalgrowth

C:\Users\Alok Kumar\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
dffeatures.head()

,weighted indegree,weighted outdegree,indegree,outdegree,Degree,pageranks,clustering,Pop Size,Area,Density,DecadalGrowth
0,81267,20700,34,34,68,0.028992,0.983066,380581,"8,249",46,6.70
1,1591890,2030004,34,34,68,0.028992,0.983066,84580777,"275,045",308,10.98
2,136010,37368,33,34,67,0.028180,0.983957,1383727,"83,743",17,25.90
3,495699,659694,34,34,68,0.028992,0.983066,31205576,"78,438",397,16.90
4,1111954,7453803,34,34,68,0.028992,0.983066,104099452,"94,163","1,102",25.10


In [17]:
dffeatures.dtypes

weighted indegree       int64
weighted outdegree      int64
indegree                int64
outdegree               int64
Degree                  int64
pageranks             float64
clustering            float64
Pop Size                int64
Area                   object
Density                object
DecadalGrowth         float64
dtype: object

In [18]:
dffeatures['Area']  =  pd.to_numeric(dffeatures['Area'].str.replace(',',''))

C:\Users\Alok Kumar\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
dffeatures['Density']  =  pd.to_numeric(dffeatures['Density'].str.replace(',',''))

C:\Users\Alok Kumar\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## 2011

In [25]:
df2011raw.columns[:29]

Index(['Area Name', 'Last residence', 'Total migrants Persons',
       'Total migrants Males', 'Total migrantsFemales',
       'Work/employment Persons', 'Work/employment Males',
       'Work/employment Females', 'Business Persons', 'Business Males',
       'Business Females', 'Education Persons', 'Education Males',
       'Education Females', 'Marriage Persons', 'Marriage Males',
       'Marriage Females', 'Moved after birth Persons',
       'Moved after birth Males', 'Moved after birth Females',
       'Moved with householdPersons', 'Moved with household Males',
       'Moved with household Females', 'Others Persons', 'Others Males',
       'Others Females', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28'],
      dtype='object')

In [29]:
top5 = df2011raw.groupby('Area Name')['Business Females'].apply(lambda grp: grp.nlargest(10)).index
top5data = []
for i in top5:
    top5data.append([df2011raw['Last residence'].loc[i[1]], i[0], df2011raw['Business Females'].loc[i[1]]])
t = pd.DataFrame(top5data, columns=['Source', 'Target','Weight'])

In [30]:
t.to_csv('E:/Github/MLN/FinalData/top10'+'BusinessFemale'+'2011.csv', index=False)

## 2001

In [31]:
df2001raw.columns[:29]

Index(['Area Name', 'Last residence', 'Total migrants Persons',
       'Total migrants Males', 'Total migrantsFemales',
       'Work/employment Persons', 'Work/employment Males',
       'Work/employment Females', 'Business Persons', 'Business Males',
       'Business Females', 'Education Persons', 'Education Males',
       'Education Females', 'Marriage Persons', 'Marriage Males',
       'Marriage Females', 'Moved after birth Persons',
       'Moved after birth Males', 'Moved after birth Females',
       'Moved with householdPersons', 'Moved with household Males',
       'Moved with household Females', 'Others Persons', 'Others Males',
       'Others Females'],
      dtype='object')

In [32]:
top5 = df2001raw.groupby('Area Name')['Business Females'].apply(lambda grp: grp.nlargest(10)).index
top5data = []
for i in top5:
    top5data.append([df2001raw['Last residence'].loc[i[1]], i[0], df2001raw['Business Females'].loc[i[1]]])
t = pd.DataFrame(top5data, columns=['Source', 'Target','Weight'])

In [33]:
t.to_csv('E:/Github/MLN/FinalData/top10'+'BusinessFemale'+'2001.csv', index=False)